# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234328164538                   -0.50    7.0
  2   -7.249895108874       -1.81       -1.39    1.0
  3   -7.251243091255       -2.87       -1.91    4.0
  4   -7.251287325745       -4.35       -2.18    3.0
  5   -7.251325135768       -4.42       -2.56    2.0
  6   -7.251334384067       -5.03       -2.83    2.0
  7   -7.251338751580       -5.36       -3.87    1.0
  8   -7.251338783387       -7.50       -3.99    4.0
  9   -7.251338798393       -7.82       -4.78    3.0
 10   -7.251338798677       -9.55       -5.36    2.0
 11   -7.251338798700      -10.63       -5.78    3.0
 12   -7.251338798704      -11.42       -6.39    2.0
 13   -7.251338798705      -12.60       -6.91    3.0
 14   -7.251338798705      -13.82       -7.42    3.0
 15   -7.251338798705      -15.05       -7.85    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.060538214591168725
[ Info: Arnoldi iteration step 2: normres = 0.5596371937201587
[ Info: Arnoldi iteration step 3: normres = 0.7169843799936884
[ Info: Arnoldi iteration step 4: normres = 0.2496845078693491
[ Info: Arnoldi iteration step 5: normres = 0.42404008615156963
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.11e-02, 6.62e-02, 3.58e-01, 2.13e-01, 4.11e-02)
[ Info: Arnoldi iteration step 6: normres = 0.30936702979332376
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.89e-03, 9.22e-02, 2.69e-01, 9.89e-02, 2.50e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09518370346477044
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.41e-04, 1.13e-02, 2.32e-02, 6.25e-02, 6.06e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1411707426654123
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (8.50e